# Homework 4 : Applied Machine Learning

In [1]:
# imports
import pandas as pd
import numpy as np
import matplotlib as plt
import sklearn

%matplotlib inline

In [12]:
# import data
df = pd.read_csv('CrowdstormingDataJuly1st.csv', index_col=0)
df.head()

,player,club,leagueCountry,birthday,height,weight,position,games,victories,ties,...,rater2,refNum,refCountry,Alpha_3,meanIAT,nIAT,seIAT,meanExp,nExp,seExp
playerShort,,,,,,,,,,,,,,,,,,,,,
lucas-wilchez,Lucas Wilchez,Real Zaragoza,Spain,31.08.1983,177.0,72.0,Attacking Midfielder,1,0,0,...,0.50,1,1,GRC,0.326391,712.0,0.000564,0.396000,750.0,0.002696
john-utaka,John Utaka,Montpellier HSC,France,08.01.1982,179.0,82.0,Right Winger,1,0,0,...,0.75,2,2,ZMB,0.203375,40.0,0.010875,-0.204082,49.0,0.061504
abdon-prats,Abdón Prats,RCD Mallorca,Spain,17.12.1992,181.0,79.0,NaN,1,0,1,...,NaN,3,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002
pablo-mari,Pablo Marí,RCD Mallorca,Spain,31.08.1993,191.0,87.0,Center Back,1,1,0,...,NaN,3,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002
ruben-pena,Rubén Peña,Real Valladolid,Spain,18.07.1991,172.0,70.0,Right Midfielder,1,1,0,...,NaN,3,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002


In [14]:
# drop rows for which we have no photo (and thus no skin rating)
df.dropna(subset=['photoID'], inplace=True)

# group by players
group_by_players = df.groupby(level=0)

# build the data used for later analysis

# labels -- TODO: determine how to attribute one label
player_rating = group_by_players.agg({'rater1':'first', 'rater2':'first'})

# data -- TODO: maybe find a way to include IAT + exp
player_description = group_by_players.agg({'height': 'mean', 
                                           'weight': 'mean',
                                           'games': 'sum',
                                           'victories': 'sum',
                                           'ties': 'sum',
                                           'defeats': 'sum',
                                           'goals': 'sum',
                                           'yellowCards': 'sum',
                                           'yellowReds': 'sum',
                                           'redCards': 'sum',
                                          })

In [15]:
player_description.isnull().sum()

games           0
height          3
goals           0
defeats         0
weight         21
redCards        0
ties            0
yellowReds      0
yellowCards     0
victories       0
dtype: int64

We can see that we still have missing values in these two fields: height and weight. We have two toptions to deal with this: either get rid of the players for which we don't have those values (in the worst case scenario, that's 24 players less) or we try to interpolate their weight and height by taking the median or the mean for example. This can introduce bias so we chose the first option

In [18]:
player_description.dropna(subset=['weight', 'height'], inplace=True)